In [1]:
import geopandas as gpd
import os
import ast
import warnings

warnings.filterwarnings("ignore")

# Base directory path
base_dir = r"C:\Users\natda\Desktop\NatDave\Academics\PhD_NU\RESEARCH\Traffic_Stress\Boston"

# Construct file paths
roads_path = os.path.join(base_dir, "street_network.shp")
junctions_path = os.path.join(base_dir, "junctions.shp")
crossings_path = os.path.join(base_dir, "crossings.shp")
stop_signs_path = os.path.join(base_dir, "osm_stops.gdb")
signals_path = os.path.join(base_dir, "osm_signals.gdb")
unassociated_signals_path = os.path.join(base_dir, "unassociated_signals.shp")
unassociated_stop_signs_path = os.path.join(base_dir, "unassociated_stop_signs.shp")

# Load shapefiles
roads = gpd.read_file(roads_path)
junctions = gpd.read_file(junctions_path)
crossings = gpd.read_file(crossings_path)
signals = gpd.read_file(signals_path)
stop_signs = gpd.read_file(stop_signs_path)

# Create 'CONTROL' column in crossings shapefile
crossings['CONTROL'] = ""

In [2]:
#########
# Signals
#########

# Reproject signals to match roads CRS
signals = signals.to_crs(roads.crs)

# Add 'ID' column to signals and assign sequential values
signals['ID'] = range(1, len(signals) + 1)


# List to track unassociated signals (initially include all)
unassociated_signals = set(signals['ID'].tolist())

# Function to calculate the distance between a signal and all junctions
def find_nearest_junction(signal, junctions_gdf, threshold=40):
    """Find the closest junction to a signal within a given threshold."""
    distances = junctions_gdf.geometry.distance(signal.geometry)
    within_threshold = distances[distances <= threshold]
    
    if within_threshold.empty:
        return None  # No junctions within threshold
    
    # Find the closest junction
    nearest_junction_idx = within_threshold.idxmin()
    nearest_junction_id = junctions_gdf.loc[nearest_junction_idx, 'JUNC_ID']
    
    return nearest_junction_id

# Iterate through each signal and find the closest junction
signal_to_junction = {}  # Dictionary to store signal-to-junction mapping

for _, signal in signals.iterrows():
    if signal['ID'] in unassociated_signals:
        nearest_junction_id = find_nearest_junction(signal, junctions)

        if nearest_junction_id:
            signal_to_junction[signal['ID']] = nearest_junction_id
            unassociated_signals.discard(signal['ID'])

# Assign "signal" control to crossings that match junctions with signals
crossings.loc[crossings['JUNC_ID'].isin(signal_to_junction.values()), 'CONTROL'] = "signal"

### Filtering signals near restricted roads
# Filter road segments where qExclude or qNoAccess is not equal to 0
restricted_roads = roads[(roads['qExclude'] != 0) | (roads['qNoAccess'] != 0)]

# Create a 50-meter buffer around restricted roads
buffered_roads = restricted_roads.buffer(50)
buffered_roads_gdf = gpd.GeoDataFrame(geometry=buffered_roads, crs=restricted_roads.crs)

# Identify signals within 50 meters of restricted roads
signals_near_restricted_roads = gpd.sjoin(signals, buffered_roads_gdf, how="inner", predicate="intersects")

# Get IDs of signals to exclude
signals_to_exclude = set(signals_near_restricted_roads['ID'])

# Remove excluded signals from the unassociated signals list
unassociated_signals -= signals_to_exclude

# Filter unassociated signals
unassociated_signals_gdf = signals[signals['ID'].isin(unassociated_signals)]

# Save the unassociated signals to a new shapefile
unassociated_signals_gdf.to_file(unassociated_signals_path, driver='ESRI Shapefile')

# Save the updated crossings shapefile with control info
crossings.to_file(crossings_path, driver='ESRI Shapefile')

# Output the number of unassociated signals
print(f"There are {len(unassociated_signals)} unassociated signals.")
print("Unassociated Signals:", list(unassociated_signals))

There are 59 unassociated signals.
Unassociated Signals: [2829, 2830, 1812, 1814, 1048, 2589, 800, 1571, 36, 548, 1572, 1068, 1069, 3138, 3139, 842, 2638, 2896, 2897, 2898, 2899, 2900, 2918, 1139, 2163, 2675, 2676, 1659, 1660, 1661, 2939, 1667, 2692, 2697, 913, 2709, 668, 2720, 3239, 172, 445, 964, 966, 967, 1230, 719, 979, 983, 2264, 2265, 1760, 1513, 1009, 1265, 499, 755, 1267, 1017, 3067]


In [3]:
############
# Stop Signs
############

# Reproject stop signs to match roads CRS
stop_signs = stop_signs.to_crs(roads.crs)

# Add 'ID' column to stop signs and assign sequential values
stop_signs['ID'] = range(1, len(stop_signs) + 1)

# Create a set of assigned stop sign IDs
assigned_stop_signs = set()

# Function to find road segments that partially fall within a 5-meter radius of a stop sign
def find_nearby_roads(stop_sign, roads_gdf, threshold=5):
    stop_buffer = stop_sign.geometry.buffer(threshold)  # Create a 5m buffer around the stop sign
    nearby_roads = roads_gdf[roads_gdf.geometry.intersects(stop_buffer)]  # Find intersecting roads

    if nearby_roads.empty:
        return None  # No road segments within the threshold

    # Find the closest intersecting road segment
    distances = nearby_roads.geometry.distance(stop_sign.geometry)
    nearest_road_idx = distances.idxmin()
    
    return nearby_roads.loc[nearest_road_idx, 'unique_id']  # Return unique_id of nearest intersecting road

# Dictionary mapping road segments to stop sign control
road_stop_control = {}

# Iterate through each stop sign and find the nearest road segment
for _, stop in stop_signs.iterrows():
    nearest_road_id = find_nearby_roads(stop, roads)
    
    if nearest_road_id:
        road_stop_control.setdefault(nearest_road_id, []).append(stop['ID'])
        assigned_stop_signs.add(stop['ID'])

# Convert 'FRM_LEG' column to a list of road segment IDs
crossings['FRM_LEG'] = crossings['FRM_LEG'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Assign "stop" control to crossings only if no "signal" exists
for idx, row in crossings.iterrows():
    frm_leg = row['FRM_LEG']  # List of road segment IDs
    
    if isinstance(frm_leg, list):
        has_stop = any(seg in road_stop_control for seg in frm_leg)
        if has_stop and row['CONTROL'] != "signal":  # Assign "stop" only if no signal exists
            crossings.at[idx, 'CONTROL'] = "stop"

# Save the updated crossings shapefile with stop sign assignments
crossings.to_file(crossings_path, driver='ESRI Shapefile')

# Identify unassociated stop signs
unassociated_stop_signs = stop_signs[~stop_signs['ID'].isin(assigned_stop_signs)]

# Save unassociated stop signs to a new shapefile
unassociated_stop_signs.to_file(unassociated_stop_signs_path, driver='ESRI Shapefile')

# Output the number of unassociated stop signs
print(f"There are {len(unassociated_stop_signs)} unassociated stop signs.")
print("Unassociated Stop Signs:", list(unassociated_stop_signs['ID']))

There are 120 unassociated stop signs.
Unassociated Stop Signs: [230, 233, 239, 356, 371, 374, 396, 397, 409, 418, 432, 434, 438, 439, 452, 476, 483, 488, 490, 500, 502, 506, 508, 510, 515, 539, 545, 547, 549, 568, 569, 582, 583, 591, 596, 597, 600, 613, 616, 618, 620, 622, 623, 624, 638, 639, 640, 643, 644, 645, 646, 647, 649, 651, 652, 653, 654, 655, 656, 660, 667, 669, 670, 671, 672, 680, 683, 685, 697, 703, 704, 705, 706, 709, 713, 723, 733, 822, 845, 886, 1023, 1061, 1071, 1148, 1182, 1246, 1249, 1277, 1296, 1336, 1384, 1418, 1443, 1450, 1494, 1511, 1515, 1516, 1517, 1518, 1519, 1520, 1521, 1561, 1582, 1604, 1605, 1625, 1636, 1646, 1656, 1672, 1673, 1689, 1690, 1701, 1713, 1714, 1715, 1716]


In [4]:
crossings['CONTROL'].value_counts()

CONTROL
          32356
signal     5865
stop       2582
Name: count, dtype: int64